In [1]:
from halotools.empirical_models import HodModelFactory
from halotools.empirical_models import TrivialPhaseSpace, ZuMandelbaum15Cens, ZuMandelbaum15Sats, \
                                        Leauthaud11Cens, Leauthaud11Sats, Zheng07Cens, Zheng07Sats, \
                                        NFWPhaseSpace, SubhaloPhaseSpace
from halotools.empirical_models import NFWPhaseSpace, SubhaloPhaseSpace, Tinker13Cens, Tinker13QuiescentSats, \
                                        TrivialProfile, Tinker13ActiveSats
from halotools_ia.ia_models.ia_model_components import CentralAlignment, RandomAlignment, RadialSatelliteAlignment, \
                                                        HybridSatelliteAlignment, MajorAxisSatelliteAlignment, SatelliteAlignment
                                                        #SubhaloAlignment
from halotools_ia.ia_models.ia_strength_models import RadialSatelliteAlignmentStrengthAlternate

from halotools_ia.ia_models.nfw_phase_space import AnisotropicNFWPhaseSpace

from intrinsic_alignments.ia_models.occupation_models import SubHaloPositions, IsotropicSubhaloPositions, SemiIsotropicSubhaloPositions

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from halotools.mock_observables import tpcf

from halotools.sim_manager import HaloTableCache
from halotools.sim_manager import CachedHaloCatalog
from halotools_ia.correlation_functions import ed_3d, ee_3d

from halotools.utils import crossmatch

import time

import emcee
from multiprocessing import Pool

import sys
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
##### FUNCTIONS

def get_coords_and_orientations(model_instance, correlation_group="all"):
    if correlation_group == "all":
        x = model_instance.mock.galaxy_table["x"]
        y = model_instance.mock.galaxy_table["y"]
        z = model_instance.mock.galaxy_table["z"]
        axis_x = model_instance.mock.galaxy_table["galaxy_axisA_x"]
        axis_y = model_instance.mock.galaxy_table["galaxy_axisA_y"]
        axis_z = model_instance.mock.galaxy_table["galaxy_axisA_z"]
        coords = np.array( [x,y,z] ).T
        orientations = np.array( [axis_x,axis_y,axis_z] ).T
        return coords, orientations, coords
    elif correlation_group == "censat":
        sat_cut = model_instance.mock.galaxy_table[model_instance.mock.galaxy_table["gal_type"]=="satellites"]
        cen_cut = model_instance.mock.galaxy_table[model_instance.mock.galaxy_table["gal_type"]=="centrals"]
        satx = sat_cut["x"]
        saty = sat_cut["y"]
        satz = sat_cut["z"]
        cenx = cen_cut["x"]
        ceny = cen_cut["y"]
        cenz = cen_cut["z"]
        axis_x = sat_cut["galaxy_axisA_x"]
        axis_y = sat_cut["galaxy_axisA_y"]
        axis_z = sat_cut["galaxy_axisA_z"]
        sat_coords = np.array( [satx,saty,satz] ).T
        cen_coords = np.array( [cenx,ceny,cenz] ).T
        sat_orientations = np.array( [axis_x,axis_y,axis_z] ).T
        return sat_coords, sat_orientations, cen_coords
    elif correlation_group == "satcen":
        sat_cut = model_instance.mock.galaxy_table[model_instance.mock.galaxy_table["gal_type"]=="satellites"]
        cen_cut = model_instance.mock.galaxy_table[model_instance.mock.galaxy_table["gal_type"]=="centrals"]
        satx = sat_cut["x"]
        saty = sat_cut["y"]
        satz = sat_cut["z"]
        cenx = cen_cut["x"]
        ceny = cen_cut["y"]
        cenz = cen_cut["z"]
        axis_x = cen_cut["galaxy_axisA_x"]
        axis_y = cen_cut["galaxy_axisA_y"]
        axis_z = cen_cut["galaxy_axisA_z"]
        sat_coords = np.array( [satx,saty,satz] ).T
        cen_coords = np.array( [cenx,ceny,cenz] ).T
        cen_orientations = np.array( [axis_x,axis_y,axis_z] ).T
        return sat_coords, cen_orientations, cen_coords
    elif correlation_group == "cencen":
        cen_cut = model_instance.mock.galaxy_table[model_instance.mock.galaxy_table["gal_type"]=="centrals"]
        cenx = cen_cut["x"]
        ceny = cen_cut["y"]
        cenz = cen_cut["z"]
        axis_x = cen_cut["galaxy_axisA_x"]
        axis_y = cen_cut["galaxy_axisA_y"]
        axis_z = cen_cut["galaxy_axisA_z"]
        cen_coords = np.array( [cenx,ceny,cenz] ).T
        cen_orientations = np.array( [axis_x,axis_y,axis_z] ).T
        return cen_coords, cen_orientations, cen_coords
    else:
        sat_cut = model_instance.mock.galaxy_table[model_instance.mock.galaxy_table["gal_type"]=="satellites"]
        satx = sat_cut["x"]
        saty = sat_cut["y"]
        satz = sat_cut["z"]
        axis_x = sat_cut["galaxy_axisA_x"]
        axis_y = sat_cut["galaxy_axisA_y"]
        axis_z = sat_cut["galaxy_axisA_z"]
        sat_coords = np.array( [satx,saty,satz] ).T
        sat_orientations = np.array( [axis_x,axis_y,axis_z] ).T
        return sat_coords, sat_orientations, sat_coords

# Eliminate halos with 0 for halo_axisA_x(,y,z)
def mask_bad_halocat(halocat):
    bad_mask = (halocat.halo_table["halo_axisA_x"] == 0) & (halocat.halo_table["halo_axisA_y"] == 0) & (halocat.halo_table["halo_axisA_z"] == 0)
    halocat._halo_table = halocat.halo_table[ ~bad_mask ]

def get_model(ind=None):
    if not ind is None:
        return models[ind]
        
    ind = model_ind[0]
    model = models[ind]
    ind += 1
    model_ind[0] = ind % len(models)
    return model
    
def get_correlation(sats_alignment, cens_alignment, correlation_group, ind):
    model_instance = get_model(ind)
    
    # Reassign a and gamma for RadialSatellitesAlignmentStrength
    model_instance.model_dictionary["satellites_orientation"].param_dict["satellite_alignment_strength"] = sats_alignment
    model_instance.model_dictionary["centrals_orientation"].param_dict["central_alignment_strength"] = cens_alignment
        
    model_instance._input_model_dictionary["satellites_orientation"].assign_satellite_orientation( table=model_instance.mock.galaxy_table )
    model_instance._input_model_dictionary["centrals_orientation"].assign_central_orientation( table=model_instance.mock.galaxy_table )
    
    # Perform correlation functions on galaxies
    coords1, orientations, coords2 = get_coords_and_orientations(model_instance, correlation_group=correlation_group)
    #galaxy_coords, galaxy_orientations = get_galaxy_coordinates_and_orientations(model_instance, halocat)
    #galaxy_omega, galaxy_eta, galaxy_xi = galaxy_alignment_correlations(galaxy_coords, galaxy_orientations, rbins)
    omega = ed_3d( coords1, orientations, coords2, rbins, period=halocat.Lbox )
    
    return omega
    
def log_prob(theta, inv_cov, x, y, halocat, rbins, split, front, correlation_group, cores):
    model_instance = get_model()
    if len(theta) == 2:
        satellite_alignment_strength, central_alignment_strength = theta
    else:
        satellite_alignment_strength = theta
        central_alignment_strength = 1

    avg_runs = 10
    omegas = []
    
    params = [ ( satellite_alignment_strength, central_alignment_strength, correlation_group, ind ) for ind in range(avg_runs) ]
    
    pool = Pool(cores)
    omegas = pool.starmap( get_correlation, params )
    
    omegas = np.array( omegas )
    omega = np.mean( omegas, axis=0 )
        
    if front:
        diff = omega[:split] - y[:split]
    else:
        diff = omega[split:] - y[split:]
    
    return -0.5 * np.dot( diff, np.dot( inv_cov, diff ) )

global_nums = []

def string_to_bool(value):
    return value == "1" or value.lower() == "true"
    
def read_variables(f_name):
    vals = {}
    f = open(f_name)
    for line in f:
        if line.strip() != '':
            key, value = [ el.strip() for el in line.split(":->:") ]
            vals[key] = value
    
    storage_location = vals["storage_location"]
    split = int( vals["split"] )
    front = string_to_bool( vals["front"] )
    correlation_group = vals["correlation_group"]
    cov_f_name = vals[ "cov_f_name" ]
    truth_f_name = vals["truth_f_name"]
    sample_name = vals["sample_name"]
    cores = int( vals["cores"] )
    
    return storage_location, split, front, correlation_group, sample_name, cov_f_name, truth_f_name, cores

def parse_args():
    job = sys.argv[1]
    variable_f_name = sys.argv[2]
    
    return job, variable_f_name

In [3]:
def axes_correlated_with_z(p, seed=None):
    r"""
    Calculate a list of 3d unit-vectors whose orientation is correlated
    with the z-axis (0, 0, 1).
    Parameters
    ----------
    p : ndarray
        Numpy array with shape (npts, ) defining the strength of the correlation
        between the orientation of the returned vectors and the z-axis.
        Positive (negative) values of `p` produce galaxy principal axes
        that are statistically aligned with the positive (negative) z-axis;
        the strength of this alignment increases with the magnitude of p.
        When p = 0, galaxy axes are randomly oriented.

    seed : int, optional
        Random number seed used to choose a random orthogonal direction

    Returns
    -------
    unit_vectors : ndarray
        Numpy array of shape (npts, 3)

    Notes
    -----
    The `axes_correlated_with_z` function works by modifying the standard method
    for generating random points on the unit sphere. In the standard calculation,
    the z-coordinate :math:`z = \cos(\theta)`, where :math:`\cos(\theta)` is just a
    uniform random variable. In this calculation, :math:`\cos(\theta)` is not
    uniform random, but is instead implemented as a clipped power law
    implemented with `scipy.stats.powerlaw`.
    """

    p = np.atleast_1d(p)
    npts = p.shape[0]

    with NumpyRNGContext(seed):
        phi = np.random.uniform(0, 2*np.pi, npts)
        # sample cosine theta nonuniformily to correlate with in z-axis
        if np.all(p == 0):
            uran = np.random.uniform(0, 1, npts)
            cos_t = uran*2.0 - 1.0
        else:
            k = alignment_strength(p)
            d = DimrothWatson()
            cos_t = d.rvs(k)

    sin_t = np.sqrt((1.-cos_t*cos_t))

    x = sin_t * np.cos(phi)
    y = sin_t * np.sin(phi)
    z = cos_t

    return np.vstack((x, y, z)).T


def axes_correlated_with_input_vector(input_vectors, p=0., seed=None):
    r"""
    Calculate a list of 3d unit-vectors whose orientation is correlated
    with the orientation of `input_vectors`.

    Parameters
    ----------
    input_vectors : ndarray
        Numpy array of shape (npts, 3) storing a list of 3d vectors defining the
        preferred orientation with which the returned vectors will be correlated.
        Note that the normalization of `input_vectors` will be ignored.

    p : ndarray, optional
        Numpy array with shape (npts, ) defining the strength of the correlation
        between the orientation of the returned vectors and the z-axis.
        Default is zero, for no correlation.
        Positive (negative) values of `p` produce galaxy principal axes
        that are statistically aligned with the positive (negative) z-axis;
        the strength of this alignment increases with the magnitude of p.
        When p = 0, galaxy axes are randomly oriented.

    seed : int, optional
        Random number seed used to choose a random orthogonal direction

    Returns
    -------
    unit_vectors : ndarray
        Numpy array of shape (npts, 3)
    """

    input_unit_vectors = normalized_vectors(input_vectors)
    assert input_unit_vectors.shape[1] == 3
    npts = input_unit_vectors.shape[0]
    
    # For some reason, this function is very sensitive, and the difference between float32 and float64 drastically changes results
    # With only a single alignment strength, the np.ones(length)*alignmnet_strength gives float64 numbers
    # but pulling the satellite_slignment_strength column from the table gives float32
    # At values of exactl 1 and -1, the float64 numbers do fine, but float32 don'table
    # Not sure why. But they do. So I put in this next line
    p = np.array(p).astype("float64")

    z_correlated_axes = axes_correlated_with_z(p, seed)

    z_axes = np.tile((0, 0, 1), npts).reshape((npts, 3))

    angles = angles_between_list_of_vectors(z_axes, input_unit_vectors)
    rotation_axes = vectors_normal_to_planes(z_axes, input_unit_vectors)
    matrices = rotation_matrices_from_angles(angles, rotation_axes)

    return rotate_vector_collection(matrices, z_correlated_axes)

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np

from halotools_ia.ia_models.ia_strength_models import alignment_strength

# vector rotations
from halotools.utils import rotate_vector_collection
from halotools.utils.mcrotations import random_perpendicular_directions, random_unit_vectors_3d
from halotools.utils.vector_utilities import (elementwise_dot, elementwise_norm, normalized_vectors,
                                              angles_between_list_of_vectors, vectors_normal_to_planes)
from halotools.utils.rotations3d import vectors_between_list_of_vectors, rotation_matrices_from_angles

# watson distribution
from watson_dist import DimrothWatson

# utilities
from warnings import warn
from astropy.utils.misc import NumpyRNGContext

from halotools.utils import crossmatch

class SubhaloAlignment(object):
    r"""
    alignment model for satellite galaxies in sub-haloes aligning with their respective subhalos
    most of the functionality here is copied from SatelltieAlignment by Duncan Campbell
    """
    def __init__(self, halocat=None, satellite_alignment_strength=1.0, prim_gal_axis='major', rotate_relative=True, **kwargs):
        r"""
        Parameters
        ----------
        satellite_alignment_strength : float
            [-1,1] bounded number indicating alignment strength

        prim_gal_axis :  string, optional
            string indicating which galaxy principle axis is correlated with the halo alignment axis.
            The options are: `major`, `intermediate`, and `minor`.
            
        rotate_relative : bool, optional
            bool indicating whether any subhalos with real_subhalo = False should be rotated to 
            preserve the relative orientation they had to their true host halo with the current host halo.
            Default is True, which will rotate the false subhalos to keep the same relative orientation
            with respect to the new host halo. A value of False will keep their original values.

        alignment_keys : list
            A list of strings indicating the keywords for the x,y- and z components of the
            halo alignment vector. Deafult is ['halo_axisA_x', 'halo_axisA_y', 'halo_axisA_z'].

        Notes
        -----
        If the kwargs or table contains a key "alignment_strength", when populating a mock,
        this will be used instead of the `satellite_alignment_stregth` parameter passed during intialization.
        This is how varying the alignment strength as a function of galaxy/halo properties is handeled.
        
        Unlike SatelliteAlignment, this alignment model needs a table to exist prior to being called. This is
        because the 
        """
        
        if halocat is None:
            msg = "halocat must be passed as an argument to preserve subhalo information"
            raise Exception(msg)
        self._halocat = halocat
        
        self._rotate_relative = rotate_relative

        self.gal_type = 'satellites'
        self._mock_generation_calling_sequence = (['assign_satellite_orientation'])

        self._galprop_dtypes_to_allocate = np.dtype(
            [(str('galaxy_axisA_x'), 'f4'), (str('galaxy_axisA_y'), 'f4'), (str('galaxy_axisA_z'), 'f4'),
             (str('galaxy_axisB_x'), 'f4'), (str('galaxy_axisB_y'), 'f4'), (str('galaxy_axisB_z'), 'f4'),
             (str('galaxy_axisC_x'), 'f4'), (str('galaxy_axisC_y'), 'f4'), (str('galaxy_axisC_z'), 'f4')])

        # specify the halo alignment vector
        if 'alignment_keys' in kwargs.keys():
            assert len(kwargs['alignment_keys'])==3
            self.list_of_haloprops_needed = kwargs['alignment_keys']
        else:
            self.list_of_haloprops_needed = ['halo_axisA_x', 'halo_axisA_y', 'halo_axisA_z']

        # set which galaxy axis is correlated with the halo alignment vector
        possible_axis = ['major', 'intermediate', 'minor']
        if prim_gal_axis in possible_axis:
            if prim_gal_axis == possible_axis[0]: self.prim_gal_axis = 'A'
            elif prim_gal_axis == possible_axis[1]: self.prim_gal_axis = 'B'
            elif prim_gal_axis == possible_axis[2]: self.prim_gal_axis = 'C'
        else:
            msg = ('`prim_gal_axis` must be one of {0}, but instead is {1}.'.format(possible_axis, prim_gal_axis))
            raise ValueError(msg)

        self._methods_to_inherit = (
            ['assign_satellite_orientation'])
        self.param_dict = ({
            'satellite_alignment_strength': satellite_alignment_strength})
    
    def _set_subhalo_values(self, **kwargs):
        r"""
        Earlier, the host halo values were input as a default for every halo within.
        Now, the satellite galaxies will have their subhalo information overwritten so all alignment will be with respect
        to the suhalo, not the host halo
        
        Parameters
        ==========
        table : astropy table holding the halo and galaxy information for every galaxy
        
        Returns
        =======
        None, table should be altered in place and will not need to return
        """
        table = kwargs['table']
        
        # Get a list of the columns shared by both the halocat and the table
        cols = list( set(self._halocat.halo_table.columns) & set(table.columns) )
        # Remove the halo_id column, otherwise there will be an issue when overwriting
        # Since this is the column used to match, there is no need to overwrite anyway
        cols.remove('halo_id')
        cols.remove('halo_hostid')              # Keep the overwritten host halo id
        
        # find where each halo_id appears in the full halocat
        halo_ids = table['halo_id']
        inds1, inds2 = crossmatch( halo_ids, self._halocat.halo_table['halo_id'] )
        
        # This overwrites the information from their host halo that was used for them instead (unless that halo is the host halo)
        for col in cols:
            table[col][ inds1 ] = self._halocat.halo_table[col][ inds2 ]
                
        # If the halo is not a real subhalo, overwrite the orientation so it is the same relative to its new host halo as its original
        if self._rotate_relative:
            self._orient_false_subhalo( table=table )
       
    def _orient_false_subhalo(self, table):
        
        # Values of interest
        axis_A = ['halo_axisA_x', 'halo_axisA_y', 'halo_axisA_z']    # Use this axis to rotate
        velocity = ['halo_vx', 'halo_vy', 'halo_vz']
        position = ['halo_x', 'halo_y', 'halo_z']
        
        mask = ( table['gal_type'] == 'satellites' ) & ( table['real_subhalo'] == False )
        
        halo_ids = table[mask]['halo_id']
        inds1, inds2 = crossmatch(halo_ids, self._halocat.halo_table["halo_id"])
        original_host_ids = self._halocat.halo_table[inds2]["halo_hostid"]
        new_host_ids = table[mask]['halo_hostid']
        
        # inds1[i] is the index in halo_ids of the halo id at position i in model_instance.mock.galaxy_table['halo_ids']
        # Similarly, inds2[i] is the index in model_instance.mock.galaxy_table['halo_ids'] of the halo id in position i in halo_ids
        #print("Lengths")
        #print( len(table['halo_id']), " -- ", len(set(table['halo_id'])) )
        #print( len(halo_ids), " -- ", len(set(halo_ids)) )
        #inds1, inds2 = crossmatch(table['halo_id'], halo_ids)
        
        halo_axesA = np.array( [ table[mask][axis_A[0]], table[mask][axis_A[1]], table[mask][axis_A[2]] ] ).T
        halo_velocities = np.array( [ table[mask][velocity[0]], table[mask][velocity[1]], table[mask][velocity[2]] ] ).T
        halo_positions = np.array( [ table[mask][position[0]], table[mask][position[1]], table[mask][position[2]] ] ).T
        
        # Get original host axes
        inds1, inds2 = crossmatch( original_host_ids, self._halocat.halo_table["halo_id"] )
        original_host_axesA = np.array( [ self._halocat.halo_table[inds2][axis_A[0]], self._halocat.halo_table[inds2][axis_A[1]], self._halocat.halo_table[inds2][axis_A[2]] ] ).T
        
        # Get new host axes
        inds1, inds2 = crossmatch(new_host_ids, self._halocat.halo_table["halo_id"])
        new_host_axesA = np.array( [ self._halocat.halo_table[inds2][axis_A[0]], self._halocat.halo_table[inds2][axis_A[1]], self._halocat.halo_table[inds2][axis_A[2]] ] ).T
    
        # Get the rotation axis to rotate original_host_axisA into new_host_axisA
        # Then use that rotation matrix to rotate halo_axisA and halo_velocities
        rot = np.array( [ self._get_rotation_matrix(original_host_axesA[i], new_host_axesA[i]) for i in range(len(original_host_axesA)) ] )
        rotated_axes = rotate_vector_collection(rot, halo_axesA )
        rotated_velocities = rotate_vector_collection(rot, halo_velocities )
        rotated_positions = rotate_vector_collection(rot, halo_positions)
        
        for i in range(len(axis_A)):
            table[ axis_A[i] ][ mask ] = rotated_axes[:,i]
            table[ velocity[i] ][ mask ] = rotated_velocities[:,i]
            table[ position[i] ][ mask ] = rotated_positions[:,i]
    
    def _get_rotation_matrix(self, a, b):
        r"""
        Returns the rotation matrix (only 3D) needed to rotate a into b
        
        Parameters
        ==========
        a : 3 element numpy array representing a vector in 3D
        b : 3 element numpy array representing a vector in 3D
        
        Returns
        =======
        mat : 3x3 numpy array representing the rotation matrix needed to rotate a in the direction of b
        """
        unit_a = normalized_vectors(a)
        unit_b = normalized_vectors(b)
        v = np.cross(unit_a,unit_b)[0]
        s = np.linalg.norm(v)                           # Sin of the angle
        c = np.dot(unit_a,unit_b.T)                  # Cos of the angle
        
        vx = np.zeros((3,3))
        vx[0,1] = -v[2]
        vx[1,0] = v[2]
        vx[0,2] = v[1]
        vx[2,0] = -v[1]
        vx[1,2] = -v[0]
        vx[2,1] = v[0]
        
        mat = np.eye(3) + vx + np.dot(vx,vx)*((1-c)/(s*s))
        return mat

    def assign_satellite_orientation(self, **kwargs):
        r"""
        Assign a set of three orthoganl unit vectors indicating the orientation
        of the galaxies' major, intermediate, and minor axis

        Parameters
        ==========
        halo_axisA_x, halo_axisA_y, halo_axisA_z :  array_like
             x,y,z components of halo alignment axis

        Returns
        =======
        major_aixs, intermediate_axis, minor_axis :  numpy nd.arrays
            arrays of galaxies' axes
        """
        
        # Before anything else, overwrite the default host halo values with the subhalo values for each satellite galaxy
        self._set_subhalo_values(table=kwargs['table'])
        
        if 'table' in kwargs.keys():
            table = kwargs['table']
            Ax = table[self.list_of_haloprops_needed[0]]
            Ay = table[self.list_of_haloprops_needed[1]]
            Az = table[self.list_of_haloprops_needed[2]]
        else:
            Ax = kwargs['halo_axisA_x']
            Ay = kwargs['halo_axisA_y']
            Az = kwargs['halo_axisA_z']

        # get alignment strength for each galaxy
        if 'table' in kwargs.keys():
            try:
                p = table['satellite_alignment_strength']
            except KeyError:
                msg = ('`satellite_alignment_strength` not detected in the table, using value in self.param_dict.')
                warn(msg)
                p = np.ones(len(Ax))*self.param_dict['satellite_alignment_strength']
        else:
            p = np.ones(len(Ax))*self.param_dict['satellite_alignment_strength']

        # set prim_gal_axis orientation
        major_input_vectors = np.vstack((Ax, Ay, Az)).T
        A_v = axes_correlated_with_input_vector(major_input_vectors, p=p)

        # randomly set secondary axis orientation
        B_v = random_perpendicular_directions(A_v)

        # the tertiary axis is determined
        C_v = vectors_normal_to_planes(A_v, B_v)

        # depending on the prim_gal_axis, assign correlated axes
        if self.prim_gal_axis == 'A':
            major_v = A_v
            inter_v = B_v
            minor_v = C_v
        elif self.prim_gal_axis == 'B':
            major_v = B_v
            inter_v = A_v
            minor_v = C_v
        elif self.prim_gal_axis == 'C':
            major_v = B_v
            inter_v = C_v
            minor_v = A_v
        else:
            msg = ('primary galaxy axis {0} is not recognized.'.format(self.prim_gal_axis))
            raise ValueError(msg)

        if 'table' in kwargs.keys():
            try:
                mask = (table['gal_type'] == self.gal_type)
            except KeyError:
                mask = np.array([True]*len(table))
                msg = ("Because `gal_type` not indicated in `table`.",
                       "The orientation is being assigned for all galaxies in the `table`.")
                print(msg)

            # check to see if the columns exist
            for key in list(self._galprop_dtypes_to_allocate.names):
                if key not in table.keys():
                    table[key] = 0.0

            # add orientations to the galaxy table
            table['galaxy_axisA_x'][mask] = major_v[mask, 0]
            table['galaxy_axisA_y'][mask] = major_v[mask, 1]
            table['galaxy_axisA_z'][mask] = major_v[mask, 2]

            table['galaxy_axisB_x'][mask] = inter_v[mask, 0]
            table['galaxy_axisB_y'][mask] = inter_v[mask, 1]
            table['galaxy_axisB_z'][mask] = inter_v[mask, 2]

            table['galaxy_axisC_x'][mask] = minor_v[mask, 0]
            table['galaxy_axisC_y'][mask] = minor_v[mask, 1]
            table['galaxy_axisC_z'][mask] = minor_v[mask, 2]

            return table
        else:
            return major_v, inter_v, minor_v


In [5]:
models = np.repeat(None, 1)
model_ind = np.array([0])
    
if __name__ == "__main__":
    job, variable_f_name =  ["0","variables/full_front1.txt"]
    #storage_location, split, front, correlation_group, sample_name, cov_f_name, truth_f_name, cores = \
    #                    read_variables( variable_f_name )
    storage_location, split, front, correlation_group, sample_name, cov_f_name, truth_f_name, cores = \
                        ["", 9, True, "all","sample_3","","",10]
    print([storage_location, split, front, correlation_group, sample_name, cov_f_name, truth_f_name, cores])

    truth_mean = np.array( truth_f_name )

    rbins = np.logspace(-1,1.4,20)
    rbin_centers = (rbins[:-1]+rbins[1:])/2.0

    cache = HaloTableCache()

    halocat = CachedHaloCatalog(simname='bolshoi', halo_finder='rockstar', redshift=0, version_name='halotools_v0p4')
    mask_bad_halocat(halocat)

    # MODELS
    cens_occ_model = Zheng07Cens()
    #cens_occ_model = Leauthaud11Cens()
    cens_prof_model = TrivialPhaseSpace()
    cens_orientation = CentralAlignment()
    prof_args = ("satellites", np.logspace(10.5, 15.2, 15))
    sats_occ_model = Zheng07Sats()
    #sats_occ_model = Leauthaud11Sats()
    sats_prof_model = SubhaloPhaseSpace(*prof_args)
    sats_orientation = SubhaloAlignment(satellite_alignment_strength=1, halocat=halocat)
    
    if sample_name == 'sample_1':
        print("A")
        cens_occ_model.param_dict['logMmin'] = 12.54
        cens_occ_model.param_dict['sigma_logM'] = 0.26

        sats_occ_model.param_dict['alpha'] = 1.0
        sats_occ_model.param_dict['logM0'] = 12.68
        sats_occ_model.param_dict['logM1'] = 13.48

        cens_orientation.param_dict['central_alignment_strength'] = 0.755
        sats_orientation.param_dict['satellite_alignment_strength'] = 0.279
    elif sample_name == 'sample_2':
        print("B")
        cens_occ_model.param_dict['logMmin'] = 11.93
        cens_occ_model.param_dict['sigma_logM'] = 0.26

        sats_occ_model.param_dict['alpha'] = 1.0
        sats_occ_model.param_dict['logM0'] = 12.05
        sats_occ_model.param_dict['logM1'] = 12.85

        cens_orientation.param_dict['central_alignment_strength'] = 0.64
        sats_orientation.param_dict['satellite_alignment_strength'] = 0.084
    elif sample_name =='sample_3':
        print("C")
        cens_occ_model.param_dict['logMmin'] = 11.61
        cens_occ_model.param_dict['sigma_logM'] = 0.26

        sats_occ_model.param_dict['alpha'] = 1.0
        sats_occ_model.param_dict['logM0'] = 11.8
        sats_occ_model.param_dict['logM1'] = 12.6

        cens_orientation.param_dict['central_alignment_strength'] = 0.57172919
        sats_orientation.param_dict['satellite_alignment_strength'] = 0.01995
    
    for i in range(len(models)):

        model_instance = HodModelFactory(centrals_occupation = cens_occ_model,
                                         centrals_profile = cens_prof_model,
                                         satellites_occupation = sats_occ_model,
                                         satellites_profile = sats_prof_model,
                                         centrals_orientation = cens_orientation,
                                         satellites_orientation = sats_orientation,
                                         model_feature_calling_sequence = (
                                         'centrals_occupation',
                                         'centrals_profile',
                                         'satellites_occupation',
                                         'satellites_profile',
                                         'centrals_orientation',
                                         'satellites_orientation')
                                        )

        print(i)
        model_instance.populate_mock(halocat, seed=132358712)
        models[i] = model_instance

    if False:
        ndim, nwalkers = 2, 5
        #ndim, nwalkers = 1, 4

        p0 = 2*((np.random.rand(nwalkers, ndim)) - 0.5)

        cov = np.load(cov_f_name)
        n = 5*5*5
        p = len(rbin_centers)

        factor = (n-p-2)/(n-1)

        if front:
            cov = cov[:split,:split]
        else:
            cov = cov[split:,split:]

        inv_cov = np.linalg.inv(cov)
        # Include the factor from the paper
        inv_cov *= factor

        try:
            f_name = os.path.join(storage_location,"MCMC_"+job+".h5")
            backend = emcee.backends.HDFBackend(f_name)
            args = [inv_cov, rbin_centers, truth_mean, halocat, rbins, split, front, correlation_group]
            moves = [emcee.moves.StretchMove(a=2),emcee.moves.StretchMove(a=1.1),emcee.moves.StretchMove(a=1.5),emcee.moves.StretchMove(a=1.3)]

            sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=args, backend=backend, moves=moves)
            sampler.run_mcmc(p0, 10000, store=True, progress=True)
            #    print(time.time()-start)

        except Exception as e:
            print(e)


['', 9, True, 'all', 'sample_3', '', '', 10]
C
0


In [6]:
print("DONE")

DONE
